In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./dataset/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./dataset/sample_submission.csv
./dataset/test.csv
./dataset/train.csv


In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torch.optim as optim
import os
import matplotlib.pyplot as plt

In [10]:
## Checking availability of GPU

if torch.cuda.is_available():
    device = torch.device(type='cuda',index=0)
else:
    device = torch.device(type='cpu',index=0)

In [11]:
device

device(type='cpu', index=0)

# **Custom classes for reading train/test data**

In [12]:
# Train -- Custom dataset
class CustomTrainDataset(Dataset):
    
    def __init__(self,path,transform):
        super().__init__()
        self.data = pd.read_csv(path,header='infer').values
        self.length = self.data.shape[0]
        self.transform = transform # image converted to pytorch tensor
        
    def __len__(self):
        return self.length
    
    def __getitem__(self,idx):
        img = self.data[idx,1:].astype(np.uint8)
        img = self.transform(np.reshape(img,(28,28,1))) # 1d to readable image
        label = self.data[idx,0]
        return img,label
    
# Test -- Custom dataset
class CustomTestDataset(Dataset):
    
    def __init__(self,path,transform):
        super().__init__()
        self.data = pd.read_csv(path,header='infer').values
        self.length = self.data.shape[0]
        self.transform = transform # image converted to pytorch tensor
        
    def __len__(self):
        return self.length
    
    def __getitem__(self,idx):
        img = self.data[idx,:].astype(np.uint8)
        img = self.transform(np.reshape(img,(28,28,1))) # 1d to readable image
        return img

In [13]:
train_data = CustomTrainDataset('./dataset/train.csv',ToTensor())
test_data = CustomTestDataset('./dataset/test.csv',ToTensor())

In [14]:
batch_size = 64

train_dataloader = DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(dataset=test_data,batch_size=batch_size)

# CNN Model****

In [15]:
class mnist_CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.relu = nn.ReLU()
        
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels = 32,
                              kernel_size=(3,3),
                              stride=1,
                              padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.mp = nn.MaxPool2d(kernel_size=(2,2),
                              stride=2,
                              padding=0)
        self.conv2 = nn.Conv2d(in_channels=32,
                              out_channels=64,
                              kernel_size=(3,3),
                              stride=1,
                              padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(in_channels=64,
                              out_channels=128,
                              kernel_size=(3,3),
                              stride=1,
                              padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(in_channels=128,
                              out_channels=200,
                              kernel_size=(3,3),
                              stride=1,
                              padding=0)
        self.bn4 = nn.BatchNorm2d(200)
        
        self.drop = nn.Dropout(0.3)
        
        self.lin1 = nn.Linear(in_features=64*200,out_features=10)
        self.bn5 = nn.BatchNorm1d(10)
        
        self.flatten = nn.Flatten()
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.drop(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.drop(x)
        x = self.mp(x)
         
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        
        x = self.drop(x)
        
        x = self.flatten(x)
        x = self.lin1(x)
        output = self.bn5(x)
        
        return output

**Training and Testing of model**

In [16]:
def train_epoch(model, train_dataloader, loss_fn, optimizer, device):
    model.train()
    track_loss = 0
    correct = 0
    
    for i, (img,label) in enumerate(train_dataloader):
        
        img, label = img.to(device), label.to(device)
        
        y_pred_train = model(img)
        loss_train = loss_fn(y_pred_train, label)
        
        track_loss += loss_train.item()
        correct += (torch.argmax(y_pred_train, dim=1)==label).type(torch.float).sum().item()
        
        loss_train.backward()
        optimizer.step()
        optimizer.zero_grad()
    epoch_loss = track_loss/len(train_dataloader)
    epoch_acc = (correct/len(train_dataloader.dataset)) * 100
    
    return epoch_loss, epoch_acc

In [17]:
model = mnist_CNN()
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()

learning_rate = 0.001

optm = optim.Adam(model.parameters(),lr = learning_rate, weight_decay=1e-5)

scheduler = optim.lr_scheduler.StepLR(optm, step_size=5, gamma=0.1)

epochs = 18

for epoch in range(epochs):
    print("Epoch no:",epoch+1)
    
    train_loss, train_acc = train_epoch(model,
                                       train_dataloader,
                                       loss_fn,
                                       optm,
                                       device)
    
    print("training: epoch loss",train_loss, "Epoch Accuracy",train_acc)
    
    print("--"*20)

Epoch no: 1
training: epoch loss 0.3551420935031304 Epoch Accuracy 96.81428571428572
----------------------------------------
Epoch no: 2
training: epoch loss 0.15239427379889575 Epoch Accuracy 98.60714285714286
----------------------------------------
Epoch no: 3
training: epoch loss 0.09830335356986868 Epoch Accuracy 98.85238095238094
----------------------------------------
Epoch no: 4
training: epoch loss 0.071147830122047 Epoch Accuracy 99.04523809523809
----------------------------------------
Epoch no: 5
training: epoch loss 0.05529971039872418 Epoch Accuracy 99.17857142857143
----------------------------------------
Epoch no: 6
training: epoch loss 0.04676989178758687 Epoch Accuracy 99.16428571428571
----------------------------------------
Epoch no: 7
training: epoch loss 0.03898860560375715 Epoch Accuracy 99.30238095238096
----------------------------------------
Epoch no: 8
training: epoch loss 0.032507344804664495 Epoch Accuracy 99.37619047619047
---------------------------

In [18]:
correct = 0
total = 0

with torch.inference_mode():
    model.eval()
    for i, (img,label) in enumerate(train_dataloader):
        
        img = img.to(device)
        label = label.to(device)
        
        pred = model(img)
        
        predicted = torch.argmax(pred,dim=1).type(torch.int).to(device)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        
train_accuracy = 100 * correct / total
print(f"Accuracy: {train_accuracy}%")

Accuracy: 99.88571428571429%


In [19]:
def evaluate_final(data, model, loss_fn, output_csv):
    model.eval()
    df = pd.read_csv(output_csv)
    
    with torch.inference_mode():
        for i,img in enumerate(data):
            img = img.to(device)
            prediction = model(img)
            
            prediction = torch.argmax(prediction,dim=1).type(torch.int).cpu()
            
            df.iloc[i*batch_size:i*batch_size + batch_size,1] = prediction.numpy()
    df.to_csv('submission.csv',index=False)
    df.head()
    
# it's time
evaluate_final(test_dataloader,
              model,
              loss_fn,
              './dataset/sample_submission.csv')